In [3]:
#importing libraries
import requests
import json
import pandas as pd
import sqlite3
from bs4 import BeautifulSoup
import pymongo

In [7]:
conn = sqlite3.connect('database.sqlite')
cur = conn.cursor()

In [17]:
cur.execute("""SELECT * from matches where season = '2011' limit 5;""").fetchall()

[(1092, 'D1', 2011, '2012-03-31', 'Nurnberg', 'Bayern Munich', 0, 1, 'A'),
 (1093, 'D1', 2011, '2011-12-11', 'Stuttgart', 'Bayern Munich', 1, 2, 'A'),
 (1094, 'D1', 2011, '2011-08-13', 'Wolfsburg', 'Bayern Munich', 0, 1, 'A'),
 (1095, 'D1', 2011, '2011-11-27', 'Mainz', 'Bayern Munich', 3, 2, 'H'),
 (1096, 'D1', 2011, '2012-02-18', 'Freiburg', 'Bayern Munich', 0, 0, 'D')]

In [31]:
cur.execute("""SELECT from matches where season = '2011' and HomeTeam is 'Bayern Munich' limit 5;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1122,D1,2011,2011-11-19,Bayern Munich,Dortmund,0,1,A
1,1178,D1,2011,2011-12-16,Bayern Munich,FC Koln,3,0,H
2,1197,D1,2011,2011-09-10,Bayern Munich,Freiburg,7,0,H
3,1211,D1,2011,2011-08-20,Bayern Munich,Hamburg,5,0,H
4,1235,D1,2011,2012-02-11,Bayern Munich,Kaiserslautern,2,0,H


In [32]:
cur.execute("""SELECT * from teams limit 5;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,Season,TeamName,KaderHome,AvgAgeHome,ForeignPlayersHome,OverallMarketValueHome,AvgMarketValueHome,StadiumCapacity
0,2017,Bayern Munich,27,26,15,597950000,22150000,75000
1,2017,Dortmund,33,25,18,416730000,12630000,81359
2,2017,Leverkusen,31,24,15,222600000,7180000,30210
3,2017,RB Leipzig,30,23,15,180130000,6000000,42959
4,2017,Schalke 04,29,24,17,179550000,6190000,62271


In [54]:
cur.execute("""select HomeTeam, 
       sum(CASE WHEN FTHG > FTAG THEN 1 ELSE 0 END) as homewin,
       sum(CASE WHEN FTHG = FTAG THEN 1 ELSE 0 END) as homedraw,
       sum(CASE WHEN FTHG < FTAG THEN 1 ELSE 0 END) as homeloss,
       sum(FTHG) as HomeGF,
       sum(FTAG) as HomeGA
       from Matches
       where Season = 2011
       group by HomeTeam
       order by HomeTeam;""")
cur.execute("""select HomeTeam, 
       sum(CASE WHEN FTHG > FTAG THEN 1 ELSE 0 END) as homewin,
       sum(CASE WHEN FTHG = FTAG THEN 1 ELSE 0 END) as homedraw,
       sum(CASE WHEN FTHG < FTAG THEN 1 ELSE 0 END) as homeloss,
       sum(FTHG) as HomeGF,
       sum(FTAG) as HomeGA
       from Matches
       where Season = 2011
       group by HomeTeam
       order by HomeTeam;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,HomeTeam,homewin,homedraw,homeloss,HomeGF,HomeGA
0,Aachen,4,6,7,15,24
1,Arsenal,12,4,3,39,17
2,Aston Villa,4,7,8,20,25
3,Augsburg,6,7,4,20,19
4,Bayern Munich,14,1,2,49,6
5,Blackburn,6,1,12,26,33
6,Bochum,7,3,7,23,23
7,Bolton,4,4,11,23,39
8,Braunschweig,6,8,3,21,15
9,Chelsea,12,3,4,41,24


In [55]:
cur.execute("""select AwayTeam, 
       sum(CASE WHEN FTHG < FTAG THEN 1 ELSE 0 END) as awaywin,
       sum(CASE WHEN FTHG = FTAG THEN 1 ELSE 0 END) as awaydraw,
       sum(CASE WHEN FTHG > FTAG THEN 1 ELSE 0 END) as awayloss,
       sum(FTHG) as AwayGF,
       sum(FTAG) as AwayGF
       from Matches
       where Season = 2011
       group by AwayTeam
       order by AwayTeam;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,AwayTeam,awaywin,awaydraw,awayloss,AwayGF,AwayGF
0,Aachen,2,7,8,23,15
1,Arsenal,9,3,7,32,35
2,Aston Villa,3,10,6,28,17
3,Augsburg,2,7,8,30,16
4,Bayern Munich,9,3,5,16,28
5,Blackburn,2,6,11,45,22
6,Bochum,3,4,10,32,18
7,Bolton,6,2,11,38,23
8,Braunschweig,4,7,6,20,16
9,Chelsea,6,7,6,22,24


In [61]:
cur.execute("""Select * from 
       (select AwayTeam,
       sum(CASE WHEN FTHG < FTAG THEN 1 ELSE 0 END) as awaywin,
       sum(CASE WHEN FTHG = FTAG THEN 1 ELSE 0 END) as awaydraw,
       sum(CASE WHEN FTHG > FTAG THEN 1 ELSE 0 END) as awayloss,
       sum(FTHG) as AwayGF,
       sum(FTAG) as AwayGF
       from Matches
       where Season = 2011
       group by AwayTeam
       order by AwayTeam)
       
       join on HomeTeam
       
       (select HomeTeam,
       sum(CASE WHEN FTHG > FTAG THEN 1 ELSE 0 END) as homewin,
       sum(CASE WHEN FTHG = FTAG THEN 1 ELSE 0 END) as homedraw,
       sum(CASE WHEN FTHG < FTAG THEN 1 ELSE 0 END) as homeloss,
       sum(FTHG) as HomeGF,
       sum(FTAG) as HomeGA
       from Matches
       where Season = 2011
       group by HomeTeam
       order by HomeTeam""")


df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

OperationalError: near ")": syntax error